<a id='Centre-Scraping-and-cleaning'></a>

In [1]:
import requests
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#from langdetect import detect

In [2]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

In [3]:
np.set_printoptions(threshold=np.inf)
pd.options.display.max_colwidth = 1000

- List of people I considered for this category

In [48]:
lista = ['Robert Kagan', 
         'Leon Kass', 
         'Jonathan Haidt', 
         'Ian Bremmer',
         'Thomas Friedman',
         'Peter Drucker', 
         'David Brooks',
         'John Avlon', 
         'Mark Satin', 
         'Arianna Huffington', 
         'Ben Bernanke',
         'Joseph Nye', 
         'Stephen Walt', 
         'Fareed Zakaria',  
         'Tibor R Machan',
         'Chris Matthews', 
         'David Rockefeller', 
         'Peter Thiel', 
         'Josh Marshall', 
         'Jeffrey D. Sachs',
         'John Rawls',
         'Edward Snowden',
         'Erik Wemple',
         'Maureen Dowd',
         'Lawrence H Summers'
]

- Getting data from goodreads

In [49]:
#Prepering the list for the goodreads format
new_lista = []
temp_lista = []

for name in lista:
    temp_lista.append(name.replace('.',''))


for n in temp_lista:
    new_lista.append(n.replace(' ','+'))
    

In [50]:
new_lista

['Robert+Kagan',
 'Leon+Kass',
 'Jonathan+Haidt',
 'Ian+Bremmer',
 'Thomas+Friedman',
 'Peter+Drucker',
 'David+Brooks',
 'John+Avlon',
 'Mark+Satin',
 'Arianna+Huffington',
 'Ben+Bernanke',
 'Joseph+Nye',
 'Stephen+Walt',
 'Fareed+Zakaria',
 'Tibor+R+Machan',
 'Chris+Matthews',
 'David+Rockefeller',
 'Peter+Thiel',
 'Josh+Marshall',
 'Jeffrey+D+Sachs',
 'John+Rawls',
 'Edward+Snowden',
 'Erik+Wemple',
 'Maureen+Dowd',
 'Lawrence+H+Summers']

In [51]:
#Scraping all the existing quotes for the person I need from Goodreads
quotes = []
author = []

for name in lista:
    second_temp_quote = []
    second_temp_author = []
    r = requests.get("https://goodquotesapi.herokuapp.com/author/{}".format(name))
    data = r.json()
    if len(data) < 1:
        pass
    else:
        for page in tqdm_notebook(range(1,data['total_pages']+1)):
            new_r = requests.get("https://goodquotesapi.herokuapp.com/author/{}?page={}".format(name,page))
            new_data = new_r.json()
            first_temp_quote = []
            first_temp_author = []

            for i in range(20):
                temp_quote = []
                temp_author = []

                try: 
                    temp_quote.append(new_data['quotes'][i]['quote'])
                    temp_author.append(new_data['quotes'][0]['author'])

                except:
                    temp_quote.append(np.nan)
                    temp_author.append(np.nan)
                first_temp_author.append(temp_author)
                first_temp_quote.append(temp_quote)
            second_temp_quote.append(first_temp_quote)
            second_temp_author.append(first_temp_author)
    quotes.append(second_temp_quote)
    author.append(second_temp_author)

In [53]:
#Creating the goodreads Dataframe
goodreads = pd.DataFrame({
    'Author': [item for l in author for sublist in l for j in l for item in sublist], 
    'Quotes': [item for l in quotes for sublist in l for j in l for item in sublist],
    'source': 'goodreads'
})

In [55]:
#Leaving every cell in a string
goodreads.Author = goodreads.Author.apply(lambda x : x[0])
goodreads.Quotes = goodreads.Quotes.apply(lambda x : x[0])

In [56]:
#Deleting all the rows where the Author is not in the list
goodreads = goodreads[goodreads['Author'].isin(lista)]

- Getting data from Wikiquote that I previously download

In [58]:
#Loading the previous dataframe scraped from Wikiquotes
wikiquotes = pd.read_csv('../WikiQuote/centre.csv')

[Wiki scraping / cleaning](../WikiQuote/Centre.ipynb#Wiki-Scraping-and-cleaning)

In [59]:
#Checking the author
wikiquotes.Author.value_counts()

Peter Drucker                85
John Rawls                   59
Thomas Friedman              31
Joseph Nye                   27
Jeffrey David Sachs          26
Mark Satin                   22
David Brooks (journalist)    21
Ian Bremmer                  20
Leon Kass                    13
Tibor R. Machan              13
Robert Kagan                 13
Jonathan Haidt               11
John Avlon                   11
Ben Bernanke                 10
Josh Marshall                 8
David Rockefeller             8
Peter Thiel                   7
Arianna Huffington            5
Stephen Walt                  3
Chris Matthews                3
Fareed Zakaria                3
Andrew Napolitano             2
Name: Author, dtype: int64

In [60]:
#Removing Author because he will go in another category
wiki_drop = (wikiquotes['Author']!= 'Andrew Napolitano')

In [61]:
wikiquotes=wikiquotes[wiki_drop]

In [62]:
import re

In [63]:
wikiquotes.Author = wikiquotes.Author.apply(lambda x : re.sub(r'\(.*?\)', '', x))

In [64]:
wikiquotes.Author.value_counts()

Peter Drucker          85
John Rawls             59
Thomas Friedman        31
Joseph Nye             27
Jeffrey David Sachs    26
Mark Satin             22
David Brooks           21
Ian Bremmer            20
Leon Kass              13
Tibor R. Machan        13
Robert Kagan           13
Jonathan Haidt         11
John Avlon             11
Ben Bernanke           10
Josh Marshall           8
David Rockefeller       8
Peter Thiel             7
Arianna Huffington      5
Stephen Walt            3
Chris Matthews          3
Fareed Zakaria          3
Name: Author, dtype: int64

In [65]:
wikiquotes = wikiquotes.drop('Political Orientation', axis=1)

In [66]:
wikiquotes['source'] = 'wikiquotes'

- Getting data from BrainQuote

In [67]:
#Creating a list for the Brainquote format
brain_lista = []
for n in lista:
    brain_lista.append(n.lower().replace(' ','_'))
    

In [68]:
brain_lista

['robert_kagan',
 'leon_kass',
 'jonathan_haidt',
 'ian_bremmer',
 'thomas_friedman',
 'peter_drucker',
 'david_brooks',
 'john_avlon',
 'mark_satin',
 'arianna_huffington',
 'ben_bernanke',
 'joseph_nye',
 'stephen_walt',
 'fareed_zakaria',
 'tibor_r_machan',
 'chris_matthews',
 'david_rockefeller',
 'peter_thiel',
 'josh_marshall',
 'jeffrey_d._sachs',
 'john_rawls',
 'edward_snowden',
 'erik_wemple',
 'maureen_dowd',
 'lawrence_h_summers']

In [69]:
#Scraping the quotes from Brainquote website
q = []
a = []

for name in brain_lista:
    temp_quote = []
    temp_author = []
    r = requests.get("https://www.brainyquote.com/authors/{}".format(name))
    soup = BeautifulSoup(r.text,'html.parser')
    for each in soup.find_all('a', attrs={'title':"view quote"}):
        temp_quote.append(each.text)
        #if len(temp_quote) > 0:
        temp_author.append(name)
    q.append(temp_quote)
    a.append(temp_author)
    


In [70]:
#Creating the Brainquote dataframe
brain = pd.DataFrame({'Author': [item for l in a for item in l],
                      'Quotes': [item for l in q for item in l],
                      'source': 'brainquote'})

In [71]:
#Removing '_' from the name and capitalize Name and Surname
brain.Author = brain.Author.apply(lambda x : x.replace('_',' ').title())

In [75]:
#Putting everything together
Centre_quotes = pd.concat([goodreads, brain, wikiquotes])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [76]:
Centre_quotes = Centre_quotes.drop_duplicates(subset='Quotes')

In [77]:
Centre_quotes.shape

(2692, 3)

- Deleting the not english quotes

In [79]:
Centre_quotes['language'] = Centre_quotes.Quotes.apply(lambda x : detect(x))

In [80]:
Centre_quotes.language.value_counts()

en    2634
es      31
it       8
fr       5
pt       2
ro       2
tl       2
af       2
ar       1
id       1
sl       1
da       1
sq       1
nl       1
Name: language, dtype: int64

In [81]:
Centre_quotes = Centre_quotes[Centre_quotes.language == 'en']

In [83]:
#Correcting few name of authors
Centre_quotes.loc[Centre_quotes['Author'].str.contains('Brooks'),'Author'] = 'David Brooks'
Centre_quotes.loc[Centre_quotes['Author'].str.contains('Sachs'),'Author'] = 'Jeffrey D. Sachs'


- Checking for quotes in quotes

In [84]:
dup = {}
#count = 0
for name in Centre_quotes.Author.unique():
    conta = []
    for text in Centre_quotes[Centre_quotes.Author==name].Quotes :
        count = 0
        for i in range(len(Centre_quotes[Centre_quotes.Author==name])):
            if text in Centre_quotes[Centre_quotes.Author==name].iloc[i,1]:
                count += 1


        conta.append(count)
    dup[name] = conta

In [85]:
Centre_quotes['Counter'] = 0
for name in Centre_quotes.Author.unique():
    print(name)
    Centre_quotes.loc[Centre_quotes.Author==name, 'Counter'] = dup[name]


Jonathan Haidt
Ian Bremmer
Peter Drucker
David Brooks
Arianna Huffington
Fareed Zakaria
Chris Matthews
David Rockefeller
Peter Thiel
Jeffrey D. Sachs
John Rawls
Edward Snowden
Maureen Dowd
Robert Kagan
Leon Kass
Thomas Friedman
John Avlon
Ben Bernanke
Joseph Nye
Mark Satin
Stephen Walt
Tibor R. Machan
Josh Marshall


In [88]:
Centre_quotes.shape

(2634, 5)

In [87]:
Centre_quotes.Counter.value_counts()

1     2212
2      269
3       95
4       30
5       15
6        7
7        3
9        1
10       1
8        1
Name: Counter, dtype: int64

In [90]:
Centre_quotes=Centre_quotes[Centre_quotes.Counter==1]

In [91]:
Centre_quotes.shape

(2212, 5)

In [92]:
Centre_quotes.drop(['source', 'language', 'Counter'], axis=1, inplace=True)

In [94]:
#Adding the political orientation
Centre_quotes['Political Orientation'] = 'Moderate'

In [95]:
#Checking
Centre_quotes.Author.value_counts()

Jonathan Haidt        503
Peter Thiel           442
Arianna Huffington    222
Peter Drucker         127
David Brooks          109
John Rawls            103
Fareed Zakaria        101
Jeffrey D. Sachs       94
Edward Snowden         63
Maureen Dowd           61
Thomas Friedman        55
David Rockefeller      49
Ian Bremmer            43
Leon Kass              39
Chris Matthews         37
Ben Bernanke           36
Joseph Nye             33
Robert Kagan           25
John Avlon             24
Mark Satin             22
Tibor R. Machan        13
Josh Marshall           8
Stephen Walt            3
Name: Author, dtype: int64

In [99]:
Centre_quotes.to_csv('centre_quotes.csv', index=False)

In [100]:
Centre_quotes.shape

(2212, 3)

In [101]:
#Selecting 4 Authors for a total of bit more than 100 quotes for the final test df
list_test = ['John Avlon', 'Robert Kagan', 'Ben Bernanke', 'Joseph Nye']

In [102]:
#Creating the Dataframe for the test
C_test = Centre_quotes[Centre_quotes['Author'].isin(list_test)]


In [103]:
C_test.Author.value_counts()


Ben Bernanke    36
Joseph Nye      33
Robert Kagan    25
John Avlon      24
Name: Author, dtype: int64

In [104]:
#removing the 4 authors from the main database
Centre_quotes = Centre_quotes[~Centre_quotes['Author'].isin(list_test)]

In [107]:
#checking everyhting is fine
Centre_quotes.Author.value_counts()

Jonathan Haidt        503
Peter Thiel           442
Arianna Huffington    222
Peter Drucker         127
David Brooks          109
John Rawls            103
Fareed Zakaria        101
Jeffrey D. Sachs       94
Edward Snowden         63
Maureen Dowd           61
Thomas Friedman        55
David Rockefeller      49
Ian Bremmer            43
Leon Kass              39
Chris Matthews         37
Mark Satin             22
Tibor R. Machan        13
Josh Marshall           8
Stephen Walt            3
Name: Author, dtype: int64

In [109]:
C_train.shape

(6057, 4)

In [110]:
#csv file for the train
Centre_quotes.to_csv('train_centre.csv', index=False)

In [106]:
#csv file for the test
C_test.to_csv('test_centre.csv', index=False)

**SECOND PART**

In [4]:
#Loading 

In [91]:
C_train = pd.read_csv('train_centre.csv')
C_test = pd.read_csv('test_centre.csv')

In [92]:
C_train.drop(C_train.index[[870, 878, 867, 311, 69, 346, 13, 183, 22, 417, 395, 466, 170, 458, 
                                      867, 878, 986, 891, 1085, 1199, 1701, 1022, 1258, 1062, 1066, 1164, 
                                      1101, 1330, 1036, 86, 206, 403, 206, 406, 122, 418, ]], inplace=True)

- Checking for quotes about the author and not from them

In [93]:
def find_autoref(df):
    autoref = {}
    for name in df.Author.unique():
        conta = []

        for text in df[df.Author==name].Quotes :
            count = 0

            if name in text or name.split(' ')[-1] in text:
                    count += 1

            conta.append(count)
        autoref[name] = conta 
    return autoref

In [94]:
autoref = find_autoref(C_train)

In [95]:
C_train['auto'] = 0
for name in C_train.Author.unique():
    print(name, ' done')
    C_train.loc[C_train.Author==name, 'auto'] = autoref[name]

Jonathan Haidt  done
Ian Bremmer  done
Peter Drucker  done
David Brooks  done
Arianna Huffington  done
Fareed Zakaria  done
Chris Matthews  done
David Rockefeller  done
Peter Thiel  done
Jeffrey D. Sachs  done
John Rawls  done
Edward Snowden  done
Maureen Dowd  done
Leon Kass  done
Thomas Friedman  done
Mark Satin  done
Stephen Walt  done
Tibor R. Machan  done
Josh Marshall  done


In [96]:
C_train.auto.sum()

85

In [97]:
C_train=C_train[C_train.auto==0]

- Checking quotes in quotes with stop words and tokenize

In [98]:
from textacy.preprocess import preprocess_text
import re
#import nltk

In [99]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(C_train)

,Author,Quotes,Political Orientation,auto


In [108]:
C_train['texicity'] = C_train.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True
                                                                     ))

In [110]:

#from nltk.corpus import stopwords


In [111]:
stops = stopwords.words('english')

In [113]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stops])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    df['texicity'] = df['texicity'].apply(lambda x: re.sub(r'\d+', '',x)) 
    return df
    

In [114]:
C_train = tok_stop(C_train)

In [115]:
fake = {}
#count = 0
for name in C_train.Author.unique():
    print(name)
    conta = []
    for j,text in enumerate(C_train[C_train.Author==name].texicity):
        print(j)
        count = 0
        for i in range(len(C_train[C_train.Author==name])):
            if text in C_train[C_train.Author==name].iloc[i,4]:
                count += 1


        conta.append(count)
    fake[name] = conta

Jonathan Haidt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273

In [59]:
C_train['rep'] = 0
for name in C_train.Author.unique():
    print(name, ' done')
    C_train.loc[C_train.Author==name, 'rep'] = fake[name]


Jonathan Haidt  done
Ian Bremmer  done
Peter Drucker  done
David Brooks  done
Arianna Huffington  done
Fareed Zakaria  done
Chris Matthews  done
David Rockefeller  done
Peter Thiel  done
Jeffrey D. Sachs  done
John Rawls  done
Edward Snowden  done
Maureen Dowd  done
Leon Kass  done
Thomas Friedman  done
Mark Satin  done
Stephen Walt  done
Tibor R. Machan  done
Josh Marshall  done


In [60]:
C_train.rep.value_counts()

1    1938
2      36
3       2
Name: rep, dtype: int64

In [61]:
C_train.to_csv('C_train_clean.csv', index=False)

In [62]:
autoref_test=find_autoref(C_test)

In [63]:
C_test['auto'] = 0
for name in C_test.Author.unique():
    print(name, ' done')
    C_test.loc[C_test.Author==name, 'auto'] = autoref_test[name]

Robert Kagan  done
John Avlon  done
Ben Bernanke  done
Joseph Nye  done


In [65]:
C_test['texicity'] = C_test.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True))

In [66]:
C_test=tok_stop(C_test)

In [68]:
fake = {}
#count = 0
for name in C_test.Author.unique():
    print(name)
    conta = []
    for i,text in enumerate(C_test[C_test.Author==name].texicity) :
        print(i)
        count = 0
        for i in range(len(C_test[C_test.Author==name])):
            if text in C_test[C_test.Author==name].iloc[i,4]:
                count += 1


        conta.append(count)
    fake[name] = conta

Robert Kagan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
John Avlon
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Ben Bernanke
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
Joseph Nye
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [69]:
C_test['rep'] = 0
for name in C_test.Author.unique():
    print(name, ' done')
    C_test.loc[C_test.Author==name, 'rep'] = fake[name]


Robert Kagan  done
John Avlon  done
Ben Bernanke  done
Joseph Nye  done


In [70]:
C_test.rep.value_counts()

1    117
2      1
Name: rep, dtype: int64

In [72]:
C_test=C_test[C_test.rep==1]

In [73]:
C_test.to_csv('C_test_clean.csv', index=False)

**Third**

- Checking for quote in the quotes and duplicate using the gram from CountVectorizer


In [117]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [119]:
def bag_word(df, orientation, mm, mx, col):
    for i in range(mm,mx):
        vect = CountVectorizer(stop_words='english', ngram_range=(i,i))
        words = {}
        lefty = df[df['Political Orientation']==orientation]
        Left_Voc = ''.join(lefty[col])
        lefty_n_grams = vect.build_analyzer()(Left_Voc)
        words['Left'] = Counter(lefty_n_grams).most_common()
        print('Number of gram: ', i)
        print(Counter(lefty_n_grams).most_common())
        print()
        print()
        return Counter(lefty_n_grams).most_common()

In [2]:
list_gram = bag_word(C_train, 'Moderate', 8,9,'texicity' )

In [122]:
list_ind=[]
for gram in list_gram:
    if gram[1]>1:
        try:
            (one,two)=(C_train.loc[C_train['texicity'].str.contains(gram[0]), 'texicity'].index)
            if len(C_train.loc[one,'texicity'])>len(C_train.loc[two,'texicity']):
                list_ind.append(two)
            else:
                list_ind.append(one)

        except:
            pass

In [3]:
l=list(set(list_ind))


In [124]:
C_train=C_train.drop(l)


In [ ]:
C_train.loc[C_train['texicity'].str.contains(gram[0]), 'texicity'].index)

In [151]:
list_ind=[]
for gram in list_gram:
    if gram[1]>1:
        try:
            (one,two)=(C_train.loc[C_train['texicity'].str.contains(gram[0][:7]), 'texicity'].index)
            if len(C_train.loc[one,'texicity'])>len(C_train.loc[two,'texicity']):
                list_ind.append(two)
            else:
                list_ind.append(one)

        except:
            pass

In [153]:
l2=list(set(list_ind))


[355, 841, 811, 314, 1004, 154, 766]

In [154]:
C_train=C_train.drop(l2)

In [164]:
#drop manually
l_3=[1586, 1335, 1377, 1006, 934]

In [165]:
C_train=C_train.drop(l_3)

In [170]:
C_train.to_csv('C_train_clean.csv')